In [1]:
import pandas as pd
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_json('sherdog/data/cumulative2.json')

In [4]:
data.iloc[0]['fighter']

{'association': None,
 'birth': None,
 'cumulative': {'loss': {'loss': {'decision': 0.0,
    'knockout': 0.0,
    'submission': 0.0,
    'total': 0.0},
   'win': {'decision': 0.0, 'knockout': 0.0, 'submission': 0.0, 'total': 0.0}},
  'win': {'loss': {'decision': 0.0,
    'knockout': 0.0,
    'submission': 0.0,
    'total': 0.0},
   'win': {'decision': 0.0,
    'knockout': 0.0,
    'submission': 0.0,
    'total': 0.0}}},
 'history': {'fights': 0.0,
  'loss': {'decision': 0.0, 'knockout': 0.0, 'submission': 0.0, 'total': 0.0},
  'positions': 0.0,
  'time': 0.0,
  'win': {'decision': 0.0, 'knockout': 0.0, 'submission': 0.0, 'total': 0.0}},
 'id': '/fighter/Chris-Write-5647',
 'nationality': None,
 'since_last_fight': 0,
 'started': 1009584000000,
 'streak': {'loss': 0.0, 'win': 0.0}}

In [18]:
def parse_date(timeinmilis):
    try:
        timestamp = timeinmilis / 1000
        return datetime.utcfromtimestamp(timestamp)
    except:
        return None

    
def flatten(data):
    X = pd.DataFrame()
    X['target'] = data['result'].map({'win': 1, 'loss': 0})
    for fighter in ['fighter', 'opponent']:
        for key in ['started']:
            curr = '{}_{}'.format(fighter, key)
            first_fight_date = data[fighter].apply(lambda x: parse_date(x[key]))
            event_date = data['date']
            X[curr] = (event_date - first_fight_date)
            X[curr] = X[curr].apply(lambda x: x.days)
            
            
        for key in ['since_last_fight']:
            curr = '{}_{}'.format(fighter, key)
            X[curr] = data[fighter].apply(lambda x: x[key])

        # Cumulative stats
        stats = 'cumulative'
        for key in ['loss', 'win']:
            for result in ['loss', 'win']:
                for method in ['decision', 'knockout', 'submission', 'total']:
                    curr = '{}_{}_{}_{}_{}'.format(fighter, stats, key, result, method)
                    X[curr] = data[fighter].apply(lambda x: x[stats][key][result][method])
                    X[curr] = X[curr].apply(lambda x: float(x))

        # Historical stats
        stats = 'history'
        for result in ['loss', 'win']:
            for method in ['decision', 'knockout', 'submission', 'total']:
                curr = '{}_{}_{}_{}'.format(fighter, stats, result, method)
                X[curr] = data[fighter].apply(lambda x: x[stats][result][method])
                X[curr] = X[curr].apply(lambda x: float(x))
        
        # Streak
        stats = 'streak'
        for result in ['loss', 'win']:
            curr = '{}_{}_{}'.format(fighter, stats, result)
            X[curr] = data[fighter].apply(lambda x: x[stats][result])
            X[curr] = X[curr].apply(lambda x: float(x))
        
        stats = 'history'
        for key in ['fights', 'positions', 'time']:
            curr = '{}_{}_{}'.format(fighter, stats, key)
            X[curr] = data[fighter].apply(lambda x: x[stats][key])
            X[curr] = X[curr].apply(lambda x: float(x))
    
    return X


def get_data(data):
    x = flatten(data)
    y = x['target']
    x.drop(['target'], axis=1, inplace=True)
    x.fillna(0, inplace=True)
    return train_test_split(x, y)

In [19]:
X_train, X_test, y_train, y_test = get_data(data)

In [42]:
X_test.iloc[50]

fighter_started                             107.0
fighter_since_last_fight                      0.0
fighter_cumulative_loss_loss_decision         0.0
fighter_cumulative_loss_loss_knockout         0.0
fighter_cumulative_loss_loss_submission       0.0
fighter_cumulative_loss_loss_total            0.0
fighter_cumulative_loss_win_decision          0.0
fighter_cumulative_loss_win_knockout          0.0
fighter_cumulative_loss_win_submission        0.0
fighter_cumulative_loss_win_total             0.0
fighter_cumulative_win_loss_decision          0.0
fighter_cumulative_win_loss_knockout          0.0
fighter_cumulative_win_loss_submission        0.0
fighter_cumulative_win_loss_total             0.0
fighter_cumulative_win_win_decision           0.0
fighter_cumulative_win_win_knockout           0.0
fighter_cumulative_win_win_submission         0.0
fighter_cumulative_win_win_total              0.0
fighter_history_loss_decision                 0.0
fighter_history_loss_knockout                 1.0


In [21]:
list(X_train.columns)

['fighter_started',
 'fighter_since_last_fight',
 'fighter_cumulative_loss_loss_decision',
 'fighter_cumulative_loss_loss_knockout',
 'fighter_cumulative_loss_loss_submission',
 'fighter_cumulative_loss_loss_total',
 'fighter_cumulative_loss_win_decision',
 'fighter_cumulative_loss_win_knockout',
 'fighter_cumulative_loss_win_submission',
 'fighter_cumulative_loss_win_total',
 'fighter_cumulative_win_loss_decision',
 'fighter_cumulative_win_loss_knockout',
 'fighter_cumulative_win_loss_submission',
 'fighter_cumulative_win_loss_total',
 'fighter_cumulative_win_win_decision',
 'fighter_cumulative_win_win_knockout',
 'fighter_cumulative_win_win_submission',
 'fighter_cumulative_win_win_total',
 'fighter_history_loss_decision',
 'fighter_history_loss_knockout',
 'fighter_history_loss_submission',
 'fighter_history_loss_total',
 'fighter_history_win_decision',
 'fighter_history_win_knockout',
 'fighter_history_win_submission',
 'fighter_history_win_total',
 'fighter_streak_loss',
 'fighter

In [46]:
input_dim = X_train.shape[1]
model = Sequential()
model.add(Dense(512, input_dim=input_dim, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [47]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(X_train.values, y_train, epochs=50, batch_size=8056)

Epoch 1/50
321460/321460 [==============================] - 9s - loss: 0.4055 - acc: 0.7922     
Epoch 2/50
321460/321460 [==============================] - 9s - loss: 0.3971 - acc: 0.7956     
Epoch 3/50
321460/321460 [==============================] - 11s - loss: 0.3979 - acc: 0.7953    
Epoch 4/50
321460/321460 [==============================] - 12s - loss: 0.4012 - acc: 0.7946    
Epoch 5/50
321460/321460 [==============================] - 12s - loss: 0.3963 - acc: 0.7966    
Epoch 6/50
321460/321460 [==============================] - 12s - loss: 0.3942 - acc: 0.7972    
Epoch 7/50
321460/321460 [==============================] - 10s - loss: 0.3966 - acc: 0.7960    
Epoch 8/50
321460/321460 [==============================] - 11s - loss: 0.3946 - acc: 0.7966    
Epoch 9/50
321460/321460 [==============================] - 12s - loss: 0.3922 - acc: 0.7978    
Epoch 10/50
321460/321460 [==============================] - 12s - loss: 0.3877 - acc: 0.7998    
Epoch 11/50
321460/321460 [==

In [61]:
scores = model.evaluate(X_test.values, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

106464/107154 [============================>.] - ETA: 0s
acc: 77.59%
